<a href="https://colab.research.google.com/github/pgbio99/Study_Gene-Expression_-Chemoresistance-in-Cancer_Machine-learning/blob/main/Annotation_GPL570.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving GPL570-55999.txt to GPL570-55999.txt
Saving Predictive_Features.csv to Predictive_Features.csv


In [ ]:
try:
  annot_df = pd.read_csv('GPL570-55999.txt', sep='\t', comment='#', header=0, usecols=['ID', 'Gene Symbol'])
  annot_df.rename(columns={'ID': 'Probe_ID', 'Gene Symbol': 'Gene_Symbol_New'}, inplace=True)

  def clean_gene_symbol(symbol):
        if pd.isna(symbol) or symbol == '---':
            return np.nan
        # Take the first symbol if multiple are present (e.g., DDR1 /// MIR4640 -> DDR1)
        return str(symbol).split(' /// ')[0].strip()

  annot_df['Gene_Symbol_New'] = annot_df['Gene_Symbol_New'].apply(clean_gene_symbol)

  predictive_df = pd.read_csv('Predictive_Features.csv')

  # 4. Merge the two DataFrames
  merged_df = pd.merge(
      predictive_df,
      annot_df,
      on='Probe_ID',
      how='left'
  )
except Exception as e:
  print(f"An error occurred: {e}")

In [ ]:
final_df = merged_df[['Probe_ID', 'Gene_Symbol_New', 'Importance']]
final_df.rename(columns={'Gene_Symbol_New': 'Gene_Symbol'}, inplace=True)

# Sort by Importance
final_df = final_df.sort_values(by='Importance', ascending=False)

In [ ]:
top_10_biomarkers = final_df.head(10)

In [ ]:
mapped_probes = final_df['Gene_Symbol'].notna().sum()
total_probes = len(final_df)
unmapped_probes = final_df['Gene_Symbol'].isna().sum()

final_output_filename = 'Top_50_Biomarkers_New_Annotation.csv'
final_df.to_csv(final_output_filename, index=False)

print("--- Annotation Status ---")
print(f"Total probes in predictive set: {total_probes}")
print(f"Probes successfully mapped to a Gene Symbol: {mapped_probes}")
print(f"Probes remaining unmapped (likely controls or non-coding): {unmapped_probes}")

print("\nTop 10 Biomarkers with Corrected Gene Annotation:")
print(top_10_biomarkers.to_markdown(index=False))

--- Annotation Status ---
Total probes in predictive set: 50
Probes successfully mapped to a Gene Symbol: 50
Probes remaining unmapped (likely controls or non-coding): 0

Top 10 Biomarkers with Corrected Gene Annotation:
| Probe_ID    | Gene_Symbol   |   Importance |
|:------------|:--------------|-------------:|
| 1007_s_at   | DDR1          |    0.0214554 |
| 201029_s_at | CD99          |    0.0178284 |
| 200872_at   | S100A10       |    0.0152589 |
| 208579_x_at | H2BFS         |    0.0138853 |
| 218450_at   | HEBP1         |    0.0136629 |
| 210749_x_at | DDR1          |    0.0134338 |
| 212077_at   | CALD1         |    0.0131842 |
| 208779_x_at | DDR1          |    0.0123741 |
| 201464_x_at | JUN           |    0.0121033 |
| 218718_at   | PDGFC         |    0.0109165 |
